In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import os, gc
import numpy as np
import pandas as pd

from sklearn import metrics, model_selection
from sklearn.preprocessing import LabelEncoder

import lightgbm as lgb

In [2]:
df = pd.read_csv('US_Accidents_Dec20.csv')

In [3]:
df1 = df

In [4]:
df1.shape

(4232541, 49)

In [5]:
df1 = df1.drop(['Distance(mi)', 'TMC', 'Country', 'Description', 'City', 'County', 'Street', 'Side', 'Zipcode', 'State', 'Airport_Code', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight'], axis=1)

In [6]:
df1.head()

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Number,...,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset
0,A-1,MapQuest,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,Night
1,A-2,MapQuest,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,2584.0,...,False,False,False,False,False,False,False,False,False,Night
2,A-3,MapQuest,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,NaN,...,False,False,False,False,False,False,False,True,False,Night
3,A-4,MapQuest,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,Night
4,A-5,MapQuest,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,NaN,...,False,False,False,False,False,False,False,True,False,Day


In [7]:
df1.isnull().sum()

ID                         0
Source                     0
Severity                   0
Start_Time                 0
End_Time                   0
Start_Lat                  0
Start_Lng                  0
End_Lat              2716477
End_Lng              2716477
Number               2687949
Timezone                4615
Weather_Timestamp      62644
Temperature(F)         89900
Wind_Chill(F)        1896001
Humidity(%)            95467
Pressure(in)           76384
Visibility(mi)         98668
Wind_Direction         83611
Wind_Speed(mph)       479326
Precipitation(in)    2065589
Weather_Condition      98383
Amenity                    0
Bump                       0
Crossing                   0
Give_Way                   0
Junction                   0
No_Exit                    0
Railway                    0
Roundabout                 0
Station                    0
Stop                       0
Traffic_Calming            0
Traffic_Signal             0
Turning_Loop               0
Sunrise_Sunset

In [8]:
df1.drop(['Timezone','Weather_Timestamp', 'Start_Time', 'End_Time', 'ID', 'Source'], axis = 1, inplace = True)

In [9]:
df1.isnull().sum()

Severity                   0
Start_Lat                  0
Start_Lng                  0
End_Lat              2716477
End_Lng              2716477
Number               2687949
Temperature(F)         89900
Wind_Chill(F)        1896001
Humidity(%)            95467
Pressure(in)           76384
Visibility(mi)         98668
Wind_Direction         83611
Wind_Speed(mph)       479326
Precipitation(in)    2065589
Weather_Condition      98383
Amenity                    0
Bump                       0
Crossing                   0
Give_Way                   0
Junction                   0
No_Exit                    0
Railway                    0
Roundabout                 0
Station                    0
Stop                       0
Traffic_Calming            0
Traffic_Signal             0
Turning_Loop               0
Sunrise_Sunset           141
dtype: int64

In [10]:
df1.dropna(axis=0, inplace=True)

In [11]:
df1.isnull().sum()

Severity             0
Start_Lat            0
Start_Lng            0
End_Lat              0
End_Lng              0
Number               0
Temperature(F)       0
Wind_Chill(F)        0
Humidity(%)          0
Pressure(in)         0
Visibility(mi)       0
Wind_Direction       0
Wind_Speed(mph)      0
Precipitation(in)    0
Weather_Condition    0
Amenity              0
Bump                 0
Crossing             0
Give_Way             0
Junction             0
No_Exit              0
Railway              0
Roundabout           0
Station              0
Stop                 0
Traffic_Calming      0
Traffic_Signal       0
Turning_Loop         0
Sunrise_Sunset       0
dtype: int64

In [12]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

In [13]:
df1['Sunrise_Sunset'] = label_encoder.fit_transform(df1['Sunrise_Sunset'])
df1['Weather_Condition'] = label_encoder.fit_transform(df1['Weather_Condition'])
df1['Wind_Direction'] = label_encoder.fit_transform(df1['Wind_Direction'])
df1.head()

,Severity,Start_Lat,Start_Lng,End_Lat,End_Lng,Number,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),...,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset
2716521,4,40.45112,-85.15048,40.35429,-85.14993,9001.0,19.9,7.3,81.0,29.85,...,False,False,False,False,False,False,False,False,False,1
2716522,4,40.35429,-85.14993,40.45112,-85.15048,473.0,19.9,7.3,81.0,29.85,...,False,False,False,False,False,False,False,False,False,1
2716526,4,40.72813,-84.78965,40.74559,-84.78962,12998.0,17.1,0.6,77.0,29.91,...,False,False,False,False,False,False,False,False,False,1
2716529,2,41.83193,-80.10143,41.84149,-80.11099,25529.0,21.0,9.9,85.0,29.69,...,False,False,False,False,False,False,False,False,False,0
2716546,2,41.48339,-81.66297,41.47692,-81.66075,3937.0,24.1,7.1,75.0,29.82,...,False,False,False,False,False,False,False,False,False,0


In [14]:
df1['Amenity'] = label_encoder.fit_transform(df1['Amenity'])
df1['Bump'] = label_encoder.fit_transform(df1['Bump'])
df1['Crossing'] = label_encoder.fit_transform(df1['Crossing'])
df1['Give_Way'] = label_encoder.fit_transform(df1['Give_Way'])
df1['Junction'] = label_encoder.fit_transform(df1['Junction'])
df1['No_Exit'] = label_encoder.fit_transform(df1['No_Exit'])
df1['Railway'] = label_encoder.fit_transform(df1['Railway'])
df1['Roundabout'] = label_encoder.fit_transform(df1['Roundabout'])
df1['Station'] = label_encoder.fit_transform(df1['Station'])
df1['Stop'] = label_encoder.fit_transform(df1['Stop'])
df1['Traffic_Calming'] = label_encoder.fit_transform(df1['Traffic_Calming'])
df1['Traffic_Signal'] = label_encoder.fit_transform(df1['Traffic_Signal'])
df1['Turning_Loop'] = label_encoder.fit_transform(df1['Turning_Loop'])
df1.head()

,Severity,Start_Lat,Start_Lng,End_Lat,End_Lng,Number,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),...,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset
2716521,4,40.45112,-85.15048,40.35429,-85.14993,9001.0,19.9,7.3,81.0,29.85,...,0,0,0,0,0,0,0,0,0,1
2716522,4,40.35429,-85.14993,40.45112,-85.15048,473.0,19.9,7.3,81.0,29.85,...,0,0,0,0,0,0,0,0,0,1
2716526,4,40.72813,-84.78965,40.74559,-84.78962,12998.0,17.1,0.6,77.0,29.91,...,0,0,0,0,0,0,0,0,0,1
2716529,2,41.83193,-80.10143,41.84149,-80.11099,25529.0,21.0,9.9,85.0,29.69,...,0,0,0,0,0,0,0,0,0,0
2716546,2,41.48339,-81.66297,41.47692,-81.66075,3937.0,24.1,7.1,75.0,29.82,...,0,0,0,0,0,0,0,0,0,0


In [15]:
df1.shape

(334821, 29)

In [16]:
X = df1.drop(['Severity'], axis = 1)

In [17]:
Y = df1['Severity']

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [19]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [20]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'poisson'},
    'num_leaves': 31,
    'learning_rate': 0.0001,
    'feature_fraction': 0.95,
    'bagging_fraction': 0.98,
    'bagging_freq': 6,
     'max_depth': -1,
     'max_bin': 511,
     'min_data_in_leaf': 20,
     'verbose':  0,
}

print('Starting training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=25,
                valid_sets=lgb_eval,
                early_stopping_rounds=20)

print('Saving model...')
# save model to file
gbm.save_model('model.txt')

print('Starting predicting...')
# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
# eval
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

Starting training...
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.213285 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[1]	valid_0's poisson: 0.516188
Training until validation scores don't improve for 20 rounds
[2]	valid_0's poisson: 0.516185
[3]	valid_0's poisson: 0.516183
[4]	valid_0's poisson: 0.516181
[5]	valid_0's poisson: 0.516179
[6]	valid_0's poisson: 0.516176
[7]	valid_0's poisson: 0.516174
[8]	valid_0's poisson: 0.516172
[9]	valid_0's poisson: 0.516169
[10]	valid_0's poisson: 0.516167
[11]	valid_0's poisson: 0.516165
[12]	valid_0's poisson: 0.516162
[13]	valid_0's poisson: 0.51616
[14]	valid_0's poisson: 0.516157
[15]	valid_0's poisson: 0.516155
[16]	valid_0's poisson: 0.516153
[17]	valid_0's poisson: 0.51615
[18]	valid_0's poisson: 0.516148
[19]	valid_0's poisson: 0.516146
[20]	valid_0's poisson: 0.516143
[21]	valid_0's poisson: 0.516141
[22]	va

In [21]:
y_pred = y_pred.round(0)
y_pred = y_pred.astype(int)
def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred)/len(y_true)
    return accuracy
Accuracy = accuracy(y_test, y_pred)

In [22]:
Accuracy


0.8594937653998357